# Import librairies

In [1]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from torch.nn import BCELoss, MSELoss, Sigmoid
from torch import optim
from torch.nn import Conv2d

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy


import warnings

warnings.filterwarnings("ignore")

sys.path.insert(0, "/appli")


plt.rcParams["figure.figsize"] = (15, 15)

from utile.pre_processing import PreProcessingPipeline
from utile.loaders import load_file, load_image, create_batch, RNSADataset
from utile.models import CustomEfficientNet
from utile.train import train_model
import cv2
from utile.plot import plot_sample

#%matplotlib widget
from opencv_transforms import transforms

seed = 53

device = "cuda" if torch.cuda.is_available() else "cpu"

print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

PyTorch Version:  1.14.0a0+410ce96
Torchvision Version:  0.15.0a0


# Import dataset

In [2]:
# Import train data
img_path = "../kaggle_dataset"
train_df = pd.read_csv("data/train.csv")

# Training procedure

## Define model

In [9]:
efficientnet = CustomEfficientNet(n_labels=1)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


# Define parameters

In [4]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "../kaggle_dataset"


# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train for
num_epochs = 1

loss = BCELoss()
# loss = MSELoss()

optimizer = optim.Adam(efficientnet.parameters(), lr=0.0001)

# Dataloader

In [5]:
transformed_dataset = RNSADataset(
    root_dir="../kaggle_dataset",
    csv_file="data/train.csv",
    transform=transforms.Compose([PreProcessingPipeline()]),
)

In [6]:
validation_split = 0.1
train_size = int((1 - validation_split) * len(transformed_dataset))
validation_size = len(transformed_dataset) - train_size
final_dataset = {}
final_dataset["train"], final_dataset["validation"] = torch.utils.data.random_split(
    transformed_dataset, [train_size, validation_size]
)

In [7]:
dataloader = {}
dataloader["train"] = DataLoader(
    final_dataset["train"], batch_size=batch_size, shuffle=True, num_workers=4
)
dataloader["validation"] = DataLoader(
    final_dataset["train"], batch_size=batch_size, shuffle=True, num_workers=4
)

# Run training

In [8]:
train_model(
    model=efficientnet,
    dataloaders=dataloader,
    criterion=loss,
    optimizer=optimizer,
    num_epochs=num_epochs,
)

0it [00:00, ?it/s]


NameError: name 'args' is not defined